In [134]:
import pandas as pd
ratings1 = pd.read_csv('data/ratings1.csv', sep=',')
ratings2 = pd.read_csv('data/ratings2.csv', sep=',')
dates = pd.read_csv('data/dates.csv', sep=',')
movies = pd.read_csv('data/movies.csv', sep=',')
orders = pd.read_csv('data/orders.csv', sep=',')

In [137]:
products = pd.read_csv('data/products.csv', sep=',')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 7, saw 2


In [28]:
ratings1_df = ratings1.copy()

In [29]:
ratings2_df = ratings2.copy()

In [30]:
dates_df = dates.copy()

In [31]:
movies_df = movies.copy()

In [ ]:
# Задание 5.2
movies_df['movieId'].nunique()
# Задание 5.3
ratings1_df['userId'].nunique()
# Задание 5.4
dates['date'] = pd.to_datetime(dates['date'])
dates['date'].dt.year.value_counts()

In [ ]:
#одинаковое количество столбцов и имена, таблица прибавляется снизу
# также замена индексов на новые, чтобы не было дублей
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
display(ratings)

In [ ]:
#удаляем дубли
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

In [ ]:
# соединяем таблицы, выводим семь последних строк
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

In [36]:
# Задание 6.3
def concat_user_files(path):
    data = pd.DataFrame()
    files = os.listdir(path)
    files.sort()
    for file in files:
        tmp_data = pd.read_csv(path + '/' + file)
        data = pd.concat([data, tmp_data])
    data = data.drop_duplicates(ignore_index=True)
    return data

In [ ]:
# объединим таблицы типом left. Так как в наших таблицах есть 
# одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки
# здесь не используется указание на столбец, с помощью которого таблицы объединяются
joined_false = ratings_dates.join(movies, rsuffix='_right', how='left')
display(joined_false)

In [75]:
joined = ratings_dates.join(movies.set_index('movieId'), on='movieId', how='left')
display(joined.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [76]:
merged = ratings_dates.merge(movies, on='movieId', how='left')
display(merged.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [ ]:
# Задание 7.5
items_df = pd.DataFrame({
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 100132, 312394], 
    'vendor': ['Samsung', 'LG', 'Apple', 'Apple', 'LG', 'Apple', 'Samsung', 'Samsung', 'LG', 'ZTE'],
    'stock_count': [54, 33, 122, 18, 102, 43, 77, 143, 60, 19]
})

purchase_df = pd.DataFrame({
    'purchase_id': [101, 101, 101, 112, 121, 145, 145, 145, 145, 221],
    'item_id': [417283, 849734, 132223, 573943, 19475, 3294095, 382043, 302948, 103845, 100132], 
    'price': [13900, 5330, 38200, 49990, 9890, 33000, 67500, 34500, 89900, 11400]
})

merged = items_df.merge(purchase_df, on='item_id', how='inner')
income = sum(merged['price']*merged['stock_count'])
display(merged)

In [49]:
#библиотека для регулярных выражений
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0,
	#очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [79]:
# Задание 8.1
joined['year_release'] = joined['title'].apply(get_year_release)
print(int(joined['year_release'].shape[0]) - int(joined['year_release'].count()))

18


In [ ]:
# Задание 8.2
print(joined[joined['year_release'] == 1999.0].groupby('title')['rating'].mean().sort_values())

In [ ]:
# Задание 8.3
print(joined[joined['year_release'] == 2010.0].groupby('genres')['rating'].mean().sort_values())

In [ ]:
# Задание 8.4
print(joined.groupby('userId')['genres'].nunique().sort_values(ascending=False))

In [ ]:
# Задание 8.5
print(joined.groupby('userId')['rating'].agg(['count', 'mean']
        ).sort_values(['count', 'mean'], ascending=[True, False]))

In [ ]:
# Задание 8.6
temp = joined[joined['year_release'] == 2018.0].groupby('genres'
        )['rating'].agg(['count', 'mean'])
result = temp[temp['count'] >10].sort_values(by = 'mean', ascending=False)
print(result)

In [130]:
# Задание 8.7
joined['year_rating'] = joined['date'].dt.year
result = joined.pivot_table(values='rating', index='year_rating', columns='genres',
                aggfunc='mean')
#print(result[result['Action|Adventure'] < 3])
#print(result['Action|Adventure|Animation|Children|Comedy|IMAX'].sort_values())
#print(result['Animation|Children|Mystery'].loc[2018])
#print(result['Comedy'])']

5.0


In [110]:
# Задание 8.9
print(joined.head())

   userId  movieId  rating                date                        title  \
0       1        1     4.0 2000-07-30 18:45:03             Toy Story (1995)   
1       1        3     4.0 2000-07-30 18:20:47      Grumpier Old Men (1995)   
2       1        6     4.0 2000-07-30 18:37:04                  Heat (1995)   
3       1       47     5.0 2000-07-30 19:03:35  Seven (a.k.a. Se7en) (1995)   
4       1       50     5.0 2000-07-30 18:48:51   Usual Suspects, The (1995)   

                                        genres  year_release  year_rating  
0  Adventure|Animation|Children|Comedy|Fantasy        1995.0         2000  
1                               Comedy|Romance        1995.0         2000  
2                        Action|Crime|Thriller        1995.0         2000  
3                             Mystery|Thriller        1995.0         2000  
4                       Crime|Mystery|Thriller        1995.0         2000  


In [ ]:
# Задание 8.10